In [1]:
import numpy as np
import skimage.color
import matplotlib.pyplot as plt
import scipy.ndimage as nd
import scipy.interpolate as inter
from scipy import ndimage
import cv2 
from tqdm import tqdm_notebook
from sklearn.cluster import MeanShift, estimate_bandwidth
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import math

In [2]:
# Load Images
short = plt.imread('shortslit.jpg')
wide = plt.imread('wideslit.jpg')
side = plt.imread('sideslit.jpg')
print(short.shape)

(2250, 4000, 3)


####  Steps
1. Invert the image for increased contrast
2. Create a binary mask with thresholding of 100. Currently based on trial and error
3. Remove smaller detected noise and blobs with erosion and dilation with kernel = (20,20)
    - Second option to remove noise: Find contours and detect the shape with the largest pixel density 
4. Measure length and width of binary mask

#### Problems
- Thresholding is not automated
- Glare/soft illumination from slit be segmented too, which is not desired.
- Segmentation of noise and reflections can be removed with erosion + dilation. However, the kernel operation on the image is currently fixed.


In [5]:
image = wide
process = 'Invert'

In [6]:
th = widgets.IntSlider(min=0, max=255, step=1, value=100,continuous_update=False)
interact(test_threshold, image = fixed(image),th = th, method = fixed('Invert'));

interactive(children=(IntSlider(value=100, continuous_update=False, description='th', max=255), Output()), _do…

In [7]:
th = widgets.IntSlider(min=0, max=255, step=1, value=100,continuous_update=False)
interact(slit_calibration, image = fixed(image), process = fixed(process), th = th)
# slit_calibration(image,process,th)

interactive(children=(IntSlider(value=100, continuous_update=False, description='th', max=255), Output()), _do…

<function __main__.slit_calibration(image, process, th)>

# Segmentation via Threhsolding ( Erosion+Dilation, Contour Isolation)

##### Isolating noise by finding the largest contour

In [8]:
th = widgets.IntSlider(min=0, max=255, step=1, value=100,continuous_update=False)
interact(slit_calibration_contour, image = fixed(image), process = fixed(process), th = th)
# slit_calibration_contour(wide,process,th)

interactive(children=(IntSlider(value=100, continuous_update=False, description='th', max=255), Output()), _do…

<function __main__.slit_calibration_contour(image, process, th)>

### Slit measured at 10mm ~ 790 pixels >>>> 1mm ~ 79 pixels

# Functions

In [1]:
def test_threshold(image, th, method):
    # Simple Thresholding Segmentation
    if process == 'Invert':
        image = image.max() -image #invert the image for max contrast 
    image = image[:,:,0] # Find red channel
    mask = image<th
    plot_compare(image,mask)
    return th


def plot_compare(img1,img2):
   # And now we show our results. 
    fig = plt.figure(figsize = (16,5))
    ax1 = fig.add_subplot(121) 
    ax1.imshow(img1, interpolation='none',  cmap='gray') 
    ax1.set_title('Before') 
#     plt.colorbar()

    ax2 = fig.add_subplot(122)
    ax2.imshow(img2, interpolation='none',  cmap='gray') 
    ax2.set_title('After') 
#     plt.colorbar()
    plt.show()


def get_dimensions(img):
    top = np.min(np.where(np.max(img,axis=1)==True))
    bottom = np.max(np.where(np.max(img,axis=1)==True))
    
    left = np.min(np.where(np.max(img,axis=0)==True))
    right = np.max(np.where(np.max(img,axis=0)==True))
    
    height = bottom - top
    width = right - left
#     print (height, 'pixels tall')
#     print (width, 'pixels wide')
    return height, width


def preprocess(image, process):
    image = image[:-70,:].copy() #crop the MicroRec label from image
    if process == 'Invert':
        image = image.max() -image #invert the image for max contrast 
#         image = skimage.color.rgb2gray(image) #isolate channel
    if process == 'Grayscale':
        image = skimage.color.rgb2gray(image) #grayscale image
    
    return image

def threshold(image, th, method):
    # Simple Thresholding Segmentation
#     if method == 'Invert':
    image = image[:,:,0] # Find red channel
    mask = image<th
    return mask

def opening_morphology(img,kernel_down=(20,20),kernel_up = (10,10)):
    erosion = ndimage.binary_erosion(img, structure=np.ones(kernel_down)).astype(img.dtype)
    dilation = ndimage.binary_dilation(erosion, structure=np.ones(kernel_up)).astype(erosion.dtype)
#     plot_compare(img,dilation)
#     return dilation
    return dilation

def slit_calibration(image,process,th):
    img = preprocess(image,process)
#     plot_compare(image,img)
    mask = threshold(img,th,process)
#     plot_compare(img,mask)
    post = opening_morphology(mask)
#     plot_compare(mask,post)
    height, width = get_dimensions(post)
    print (height, 'pixels tall')
    print (width, 'pixels wide')
    plot_compare(image,post)
#     plot_compare(image,post)

In [2]:
def slit_calibration_contour(image,process,th): 
    img = preprocess(image,process)
    mask = threshold(img,th,process)
    sample = mask.copy()
    contours, hierarchy = cv2.findContours(sample.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    number_of_objects_in_image= len(contours)
#     print ("The number of objects in this image: ", str(number_of_objects_in_image))


    display = image.copy()
    pixel_densities = []
    total = 0
    # Iterate through each contour and append to pixel_densities, create display image where each contour is located
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        mask2 = np.zeros(display.shape, dtype=np.uint8)
        cv2.fillPoly(mask2, [c], [255,255,255])
        mask2 = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY)
        pixels = cv2.countNonZero(mask2)
        total += pixels
        pixel_densities.append(pixels)
#         cv2.putText(display, '{}'.format(pixels), (x,y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

#     print(total, 'total segmented pixels.')

    # Identify the maximum contour and its position
    max_index = pixel_densities.index(max(pixel_densities))
    print('maximum contour area at index',max_index, 'with pixel area of',max(pixel_densities))
    mask2 = np.zeros(display.shape, dtype=np.uint8)
    cv2.fillPoly(mask2, [contours[max_index]], [255,255,255])
    mask2 = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY) # Create a unique mask containing only this contour 

    # Define the bounding rectangle of the largest contour in the mask
    x,y,w,h = cv2.boundingRect(contours[max_index])

    #Calculate rotation of slit
    rect = cv2.minAreaRect(contours[max_index])
    print('Segmented slit is angled by: ',math.floor(rect[2]), 'degrees')
#     box = cv2.boxPoints(rect)
#     draw = image.copy()
#     box = np.int0(box)
#     bound = cv2.drawContours(draw,[box],0,(255,255,255),2)

    # fig = plt.figure(figsize = (30,10))
#     plt.imshow(mask2, cmap = 'gray')
    
    if w>h:
        print('Horizontal Slit')
        kernel = np.ones((10,10),np.uint8)
    else:
        print('Vertical Slit')
        kernel = np.ones((10,10),np.uint8)

    dilated_img = cv2.erode(mask2, kernel)
    
    height, width = get_dimensions(dilated_img/255)
#     title = 'Vertical Slit: ' + str(height) + ' pixels tall, ' + str(width) +' pixels wide'
#     plt.title(title)
#     plt.imshow(dilated_img,cmap = 'gray')
    
    print(
#         '\n','x position:',x,'\n',
#           'y position:',y,'\n',
          'width w:',width,'\n',
          'height h:',height,'\n')
    
    plot_compare(image,dilated_img)